In [3]:
import pandas as pd
import numpy as np

import spacy
from spacy.tokenizer import Tokenizer

from sklearn.externals import joblib 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

# nlp = spacy.load("en_core_web_lg")

In [4]:
data = pd.read_csv('../DS_strain_data.csv', na_values='None')
# data = data.set_index('name')
print(data.shape)
data.head()

(2349, 11)


,Unnamed: 0,Id,Strain,Race,Flavors,Positive,Negative,Medical,Rating,Description,everything
0,0,0,100 Og,hybrid,"Earthy, Sweet, Citrus","Creative, Energetic, Tingly, Euphoric, Relaxed","Dizzy, Dry Mouth, Paranoid, Anxious","Depression, Stress, Fatigue",4.0,$100 OG is a 50/50 hybrid strain that packs a ...,"Earthy, Sweet, Citrus,Creative, Energetic, Tin..."
1,1,1,1024,sativa,"Spicy/Herbal, Sage, Woody","Uplifted, Happy, Relaxed, Energetic, Creative","Dizzy, Dry Mouth, Paranoid, Anxious","Depression, Stress, Fatigue",4.4,1024 is a sativa-dominant hybrid bred in Spain...,"Spicy/Herbal, Sage, Woody,Uplifted, Happy, Rel..."
2,2,2,13 Dawgs,hybrid,"Apricot, Citrus, Grapefruit","Tingly, Creative, Hungry, Relaxed, Uplifted","Dizzy, Dry Mouth, Paranoid, Anxious","Depression, Stress, Fatigue",4.2,13 Dawgs is a hybrid of G13 and Chemdawg genet...,"Apricot, Citrus, Grapefruit,Tingly, Creative, ..."
3,3,3,24K Gold,hybrid,"Citrus, Earthy, Orange","Happy, Relaxed, Euphoric, Uplifted, Talkative","Dizzy, Dry Mouth, Paranoid, Anxious","Depression, Stress, Fatigue",4.6,"Also known as Kosher Tangie, 24k Gold is a 60%...","Citrus, Earthy, Orange,Happy, Relaxed, Euphori..."
4,4,4,3 Bears Og,indica,"Sweet, Pungent, Earthy","Relaxed, Happy, Sleepy, Creative, Euphoric","Dizzy, Dry Mouth, Paranoid, Anxious","Depression, Stress, Fatigue",4.4,3 Bears OG by Mephisto Genetics is an autoflow...,"Sweet, Pungent, Earthy,Relaxed, Happy, Sleepy,..."


In [5]:
from gensim.utils import simple_preprocess

def tokenize(text):
    return [token for token in simple_preprocess(text)]

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words = 'english',
                      # ngram_range=(1,2),
                      # max_df=.5,
                      # min_df=3,
                       tokenizer= tokenize)

dtm = tfidf.fit_transform(data['everything'])
                       
dtm = pd.DataFrame(dtm.todense(), columns = tfidf.get_feature_names())
                       
dtm.head()

,ammonia,anxious,appetite,apple,apricot,aroused,berry,blue,blueberry,butter,...,tar,tea,tingly,tobacco,tree,tropical,uplifted,vanilla,violet,woody
0,0.0,0.161923,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.448640,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
1,0.0,0.115515,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.156295,0.0,0.0,0.350248
2,0.0,0.103597,0.0,0.0,0.614224,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.287038,0.0,0.0,0.0,0.140171,0.0,0.0,0.000000
3,0.0,0.143521,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.194189,0.0,0.0,0.000000
4,0.0,0.177011,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000


In [7]:
from sklearn.neighbors import NearestNeighbors

nn = NearestNeighbors(n_neighbors=5, algorithm='kd_tree')
nn.fit(dtm)

NearestNeighbors(algorithm='kd_tree', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=None, n_neighbors=5, p=2, radius=1.0)

In [8]:
data.iloc[552]

Unnamed: 0                                                   552
Id                                                           552
Strain                                                    Citrix
Race                                                      hybrid
Flavors                        Citrus, Grapefruit, Orange, Lemon
Positive       Euphoric, Happy, Energetic, Uplifted, Aroused,...
Negative                            Dry Mouth, Paranoid, Anxious
Medical        Depression, Stress, Lack of Appetite, Nausea, ...
Rating                                                       4.3
Description    Citrix is a hybrid cannabis strain that draws ...
everything     Citrus, Grapefruit, Orange, Lemon,Euphoric, Ha...
Name: 552, dtype: object

In [9]:
nn.kneighbors([dtm.iloc[100]])  # The second set of values are the suggested strains.

(array([[0.        , 0.54563368, 0.66674909, 0.66964119, 0.67499652]]),
 array([[ 100, 1677, 1752, 2339,  506]]))

In [10]:
test_phrase = ["""
I want something earthy Euphoric dizzy Depression
"""]

In [11]:
new = tfidf.transform(test_phrase)

In [12]:
nn.kneighbors(new.todense())  # The second set of values are the suggested strains.

(array([[0.92737593, 0.98637406, 0.99283283, 1.00028364, 1.00028364]]),
 array([[ 530,  138, 2271,  428, 1587]]))

In [13]:
from sklearn.externals import joblib 
joblib.dump(tfidf, 'baseline_tfidf.pkl')

['baseline_tfidf.pkl']

In [43]:
joblib.dump(nn, 'baseline_model.pkl')

['baseline_model.pkl']

In [50]:
vectorized = tfidf.transform
joblib.dump(vectorized, 'vectorized.pkl')

['vectorized.pkl']

In [51]:
kneighbors = nn.kneighbors
joblib.dump(kneighbors, 'kneighbors.pkl')

['kneighbors.pkl']